In [3]:
%matplotlib widget

In [4]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [5]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [6]:
gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx']
ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx']

In [7]:
folder = 'data'
prefix = 'nagdong2'
def save_gan_data(gan_dfs, ori_dfs, gan_files, ori_files, train_mean, train_std, prefix):
    for i in range(len(gan_dfs)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs[i].to_excel(path, index=False)
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs[i].to_excel(path, index=False)
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std.to_frame(name='std').to_excel(path)
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean.to_frame(name='mean').to_excel(path)

def load_gan_data(gan_files, ori_files, prefix):
    gan_dfs = []
    ori_dfs = []
    for i in range(len(gan_files)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs.append(pd.read_excel(path))
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs.append(pd.read_excel(path))
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std = pd.read_excel(path, index_col=0).loc[:, 'std']
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean = pd.read_excel(path, index_col=0).loc[:, 'mean']
    return gan_dfs, ori_dfs, train_mean, train_std

In [8]:
gan_dfs, ori_dfs, train_mean, train_std = load_gan_data(gan_files, ori_files, prefix=prefix)
for i in range(len(gan_dfs)):
    gan_dfs[i].columns = gan_dfs[i].columns+'%d'%(i+1)
    ori_dfs[i].columns = ori_dfs[i].columns+'%d'%(i+1)

data/도개_gan.xlsx
data/신암_gan.xlsx
data/회상_gan.xlsx


In [9]:
file_names = []
file_names = ['보_구미보_2016.xlsx', '보_구미보_2017.xlsx', '보_구미보_2018.xlsx', '보_구미보_2019.xlsx']
folder = 'data'
bo_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    bo_dfs.append(pd.read_excel(path).iloc[:, 2:4])
bo_df = pd.concat(bo_dfs, axis=0)

bo_mean = bo_df.mean()
bo_std = bo_df.std()
bo_df = (bo_df - bo_mean)/bo_std

bo_df.reset_index(drop=True, inplace=True)

data/보_구미보_2016.xlsx
data/보_구미보_2017.xlsx
data/보_구미보_2018.xlsx
data/보_구미보_2019.xlsx


In [10]:
file_names = []
file_names = ['AWS_옥산_2016.xlsx', 'AWS_옥산_2017.xlsx', 'AWS_옥산_2018.xlsx', 'AWS_옥산_2019.xlsx']
folder = 'data'
aws_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    aws_dfs.append(pd.read_excel(path).iloc[:, -4:])
aws_df = pd.concat(aws_dfs, axis=0)

aws_mean = aws_df.mean()
aws_std = aws_df.std()
aws_df = (aws_df - aws_mean)/aws_std
aws_df.reset_index(drop=True, inplace=True)

data/AWS_옥산_2016.xlsx
data/AWS_옥산_2017.xlsx
data/AWS_옥산_2018.xlsx
data/AWS_옥산_2019.xlsx


In [11]:
folder = 'data'
path = os.path.join(folder, 'date.xlsx')
date_df = pd.read_excel(path, index_col='ymdh')

In [12]:
file_names = [
              ['상주1_2016.xlsx',
               '상주1_2017.xlsx',
               '상주1_2018.xlsx',
               '상주1_2019.xlsx'],
              ['영강2-1_2016.xlsx',
               '영강2-1_2017.xlsx',
               '영강2-1_2018.xlsx',
               '영강2-1_2019.xlsx'],
             ]
folder = 'water_data/낙동강2/수질측정망'

filename = '상주1_2016.xlsx'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
#df = df.loc[:, qm_cols]

qm_dfs = []
qm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, qm_cols]
    qm_dfs.append(pd.concat(dfs))
    qm_dfs[j] = pd.concat([date_df, qm_dfs[j]], axis=1)
    qm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        qm_dfs[j].loc[0, col] = qm_dfs[j].loc[first_series[col], col]
        qm_dfs[j].loc[len(qm_dfs[j])-1, col] = qm_dfs[j].loc[last_series[col], col]
    
    qm_dfs[j] = qm_dfs[j].interpolate(method='pchip')

In [13]:
qm_mean = []
qm_std = []

for i in range(len(qm_dfs)):
    qm_dfs[i].columns=qm_dfs[i].columns+'%d'%(i+1)
    qm_mean.append(qm_dfs[i].mean())
    qm_std.append(qm_dfs[i].std())
    qm_dfs[i] = (qm_dfs[i]-qm_mean[i])/qm_std[i]

In [14]:
mean = pd.concat([train_mean, bo_mean, aws_mean]+qm_mean)
std = pd.concat([train_std, bo_std, aws_std]+qm_std)
mean

tmpr_value       1.619773e+01
ph_value         7.957707e+00
do_value         1.028830e+01
ec_value         2.361925e+02
toc_value        2.949259e+00
총질소_값            2.511900e+00
총인_값             1.923296e-02
클로로필-a_값         1.821510e+01
Day sin          6.037103e-15
Day cos          2.747682e-15
Year sin        -5.681266e-08
Year cos         2.053380e-05
upstrim_wlv      3.246092e+03
dwstrm_wlv       2.571552e+03
rn60m_value      7.638889e-02
ta_value         8.758325e+00
hm_value         2.880299e+01
ps_value         7.607794e+02
nh3n1            7.441017e-02
no3n1            1.995208e+00
ph1              7.986643e+00
cod1             5.426084e+00
ec1              2.315260e+02
bod1             1.595262e+00
ss1              6.677707e+00
do1              1.144842e+01
tn1              2.423511e+00
tp1              2.772457e-02
chlorophylla1    1.183009e+01
toc1             2.633367e+00
nh3n2            2.100124e-01
no3n2            1.963437e+00
ph2              7.703737e+00
cod2      

**Feature selection**

In [15]:
#ori_df = pd.concat(ori_dfs, axis=1)
#gan_df = pd.concat(gan_dfs, axis=1)
#ori_df = pd.concat(ori_dfs+[bo_df, aws_df]+qm_dfs, axis=1)
#gan_df = pd.concat(gan_dfs+[bo_df, aws_df]+qm_dfs, axis=1)
#ori_df = pd.concat(ori_dfs+[qm_dfs[1]], axis=1)
#gan_df = pd.concat(gan_dfs+[qm_dfs[1]], axis=1)
ori_df = pd.concat(ori_dfs+[bo_df, aws_df], axis=1)
gan_df = pd.concat(gan_dfs+[bo_df, aws_df], axis=1)

In [16]:
#ori_df = pd.concat(ori_dfs, axis=1)
#gan_df = pd.concat(gan_dfs, axis=1)

#ori_df = pd.concat(ori_dfs+[bo_df, aws_df]+qm_dfs, axis=1)
#gan_df = pd.concat(gan_dfs+[bo_df, aws_df]+qm_dfs, axis=1)

total_no = ori_df.shape[0]
train_no = int(total_no*0.8)
val_no = int(total_no*0.1)

train_slice = slice(0, train_no)
#val_slice = slice(train_no, train_no+val_no)
#val_slice = slice(train_no, None)
test_slice = slice(train_no, None)
#test_slice = slice(train_no+val_no, None)
#test_slice = slice(train_no+val_no, None)
val_slice = slice(train_no+val_no, None)

train_df = pd.DataFrame(gan_df[train_slice])
val_df = pd.DataFrame(gan_df[val_slice])
test_df = pd.DataFrame(gan_df[test_slice])

train_ori_df = pd.DataFrame(ori_df[train_slice])
val_ori_df = pd.DataFrame(ori_df[val_slice])
test_ori_df = pd.DataFrame(ori_df[test_slice])

num_features = train_df.shape[1]
num_features

34

In [17]:
gan_hr_df = gan_df.groupby(gan_df.index //24).mean()
total_no = gan_hr_df.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_day_df = gan_hr_df[train_slice]
val_day_df = gan_hr_df[val_slice]
test_day_df = gan_hr_df[test_slice]

**OUT FEATURE**

In [18]:
out_features = [7]
out_num_features=len(out_features)
input_days = 10
out_feature_name = train_df.columns[out_features[0]]

**Data Generator**

In [19]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [20]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [21]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [22]:
@property
def train(self):
  return self.make_dataset(self.train_df, train=True)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [23]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        #print('')
        #print('self.total_no =', self.total_no)
        #print('')
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return self.no//self.batch_size
        #return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('')
        #print('self.no =', self.no, 'self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2]
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            
            x = np.append(x, [X], axis = 0)
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

In [24]:
def make_dataset_water(self, data, train=False):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      #batch_size = 128 if train else data.shape[0]-(self.input_width+self.label_width),
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  #return ds.batch(batch_size=128)
  if train:
      #return ds.repeat(10).prefetch(3)
      return ds.repeat(-1).prefetch(5)
  else:
      return ds.prefetch(5)

WindowGenerator.make_dataset = make_dataset_water

In [25]:
def plot2(self, model=None, plot_col=0, max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  #plot_out_col_index = self.column_indices[plot_out_col]
  out_features_np = np.array(out_features)
  plot_out_col_index = np.argwhere(out_features_np == plot_col_index)[0][0]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.plot(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.plot(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [26]:
train_df.columns

Index(['tmpr_value1', 'ph_value1', 'do_value1', 'ec_value1', 'toc_value1',
       '총질소_값1', '총인_값1', '클로로필-a_값1', 'Day sin1', 'Day cos1', 'Year sin1',
       'Year cos1', 'tmpr_value2', 'ph_value2', 'do_value2', 'ec_value2',
       'toc_value2', '총질소_값2', '총인_값2', '클로로필-a_값2', 'tmpr_value3',
       'ph_value3', 'do_value3', 'ec_value3', 'toc_value3', '총질소_값3', '총인_값3',
       '클로로필-a_값3', 'upstrim_wlv', 'dwstrm_wlv', 'rn60m_value', 'ta_value',
       'hm_value', 'ps_value'],
      dtype='object')

In [27]:
OUT_STEPS = 24*5
multi_window = WindowGenerator(input_width=24*input_days,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

#multi_window.plot2(plot_col=out_features[0])
#multi_window.plot2(plot_col='클로로필-a_값1', plot_out_col='클로로필-a_값1')
multi_window.plot2(plot_col='클로로필-a_값1')
multi_window

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total window size: 360
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 

In [ ]:
'''
OUT_STEPS = 5
multi_window = WindowGenerator(input_width=input_days,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_day_df,
                               val_df=val_day_df,
                               test_df=test_day_df,
                               )

#multi_window.plot2(plot_col=out_features[0])
#multi_window.plot2(plot_col='클로로필-a_값1', plot_out_col='클로로필-a_값1')
multi_window.plot2(plot_col='클로로필-a_값1')
multi_window
'''

In [28]:
multi_val_performance = {}
multi_performance = {}

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

#multi_val_performance = {}
#multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test)
print('val performance =', multi_val_performance['Last'])
print('test performance = ', multi_performance['Last'])
multi_window.plot2(last_baseline, plot_col=train_df.columns[out_features[0]])

In [31]:
#MAX_EPOCHS = 400
MAX_EPOCHS = 20

def nse(y_true, y_pred):
    mean = tf.reduce_mean(y_true)
    return 1. - tf.reduce_sum(tf.square(y_true-y_pred))/tf.reduce_sum(tf.square(y_true-mean))

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError(), nse])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS, steps_per_epoch=10,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [ ]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
#multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
#multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=1)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

In [ ]:
len(train_df)//128

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_linear_model, plot_col=out_feature_name)

In [ ]:
58*24*10

In [ ]:
10240/2

In [ ]:
MAX_EPOCHS = 40 
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Flatten(),
    # Shape => [batch, 1, dense_units]
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(8192, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['Dense'])
print('test performance = ', multi_performance['Dense'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_dense_model, plot_col=out_feature_name)

In [ ]:
MAX_EPOCHS = 5 
#CONV_LAYER_NO = 3
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    keras.layers.Conv1D(256, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(256, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Conv1D(512, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(512, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(512, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Conv1D(1024, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(1024, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(1024, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Conv1D(2048, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(2048, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(2048, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    #keras.layers.MaxPooling1D(padding='same'),
    
    #keras.layers.Conv1D(4096, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    #keras.layers.Activation('relu'),
    #keras.layers.Conv1D(4096, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    #keras.layers.Activation('relu'),
    #keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Flatten(),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Dropout(0.5),
    
    # Shape => [batch, 1,  out_steps*features]
    #tf.keras.layers.Dense(OUT_STEPS*num_features,
                          #kernel_initializer=tf.initializers.zeros),
    tf.keras.layers.Dense(OUT_STEPS*num_features),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])



In [ ]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
plot_history(history)

**GRU model**  
best: all feature, 10 days, 256,256
```
val performance = [0.46297532320022583, 0.4968714416027069]
test performance =  [0.5573816895484924, 0.5664846301078796]
```

In [79]:
#MAX_EPOCHS = 150
MAX_EPOCHS = 1 
multi_gru_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.GRU(256, return_sequences=False, dropout=0.1, recurrent_dropout=0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_gru_model, multi_window)
#multi_gru_model.load_weights('nagdong_cl.h5')

#IPython.display.clear_output()

multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val)
multi_performance['GRU'] = multi_gru_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

51/51 [==============================] - 8s 155ms/step - loss: 0.4113 - mean_absolute_error: 0.4467 - nse: 0.5090
val performance = [0.486502081155777, 0.5207767486572266, 0.5891494750976562]
test performance =  [0.41129836440086365, 0.4467115104198456, 0.5090007781982422]


In [ ]:
multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val)

In [66]:
multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val)
multi_performance['GRU'] = multi_gru_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

51/51 [==============================] - 8s 153ms/step - loss: 0.4123 - mean_absolute_error: 0.4473 - nse: 0.5066
val performance = [0.4906741678714752, 0.5226204991340637, 0.5850632786750793]
test performance =  [0.4123188555240631, 0.4473099112510681, 0.5066457390785217]


In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_gru_model, plot_col=out_feature_name)

In [76]:
multi_gru_model.save_weights('nagdong_cl.h5')

In [ ]:
#MAX_EPOCHS = 150
MAX_EPOCHS = 5
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    #tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    #tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.1, recurrent_dropout=0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_lstm_model, plot_col=out_feature_name)

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_gru_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

In [34]:
from tqdm import tqdm

class DaysPrediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=24*input_days,
                 out_steps=OUT_STEPS,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        # fix해야함
        #len_tt = df.shape[0]//in_steps*in_steps
        #self.n_days = (len_tt - in_steps)//24
        self.n_days = df.shape[0]//24
        self.pred_days = out_steps//24
        self.in_days = in_steps//24
        self.gt = np.full((self.n_days), np.nan)
        for i in range(self.n_days):
            x = self.df[i*24:(i*24+24)].to_numpy()
            #x = val_df[i*24:(i*24+24)].to_numpy()
            self.gt[i] = np.average(x[0:24, out_features[0]])
        
    def predict(self, model):
        preds = np.full((self.pred_days, self.n_days), np.nan)
        for i in tqdm(range(self.n_days-self.in_days)):
            x = self.df[i*24:(i*24+self.in_steps)].to_numpy()
            #x = val_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            y = model.predict(x)
            for j in range(self.pred_days):
                if i+self.in_days+j >= self.n_days:
                    break
                preds[j][i+self.in_days+j] = np.average(y[:, j*24:j*24+24, :])
        return preds

In [35]:
class Prediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=input_days,
                 out_steps=OUT_STEPS,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        len_tt = df.shape[0]
        #self.n_days = len_tt - in_steps
        self.n_days = len_tt
        self.pred_days = out_steps
        self.in_days = in_steps
        self.gt = self.df.iloc[:, out_features[0]].to_numpy()
        
    def predict(self, model):
        preds = np.full((self.pred_days, self.n_days), np.nan)
        for i in tqdm(range(self.n_days-self.in_days)):
        #for i in tqdm(range(1)):
            x = self.df[i:(i+self.in_steps)].to_numpy()
            #x = val_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            y = model.predict(x)
            #print(y.shape)
            for j in range(self.pred_days):
                if i+self.in_days+j >= self.n_days:
                    break
                preds[j][i+self.in_days+j] = y[0, j, 0]
        return preds

In [36]:
class HoursPrediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=24*input_days,
                 out_steps=OUT_STEPS,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        # fix해야함
        #len_tt = df.shape[0]//in_steps*in_steps
        #self.n_days = (len_tt - in_steps)//24
        self.n_days = df.shape[0]//24
        self.pred_days = out_steps//24
        self.in_days = in_steps//24
        self.gt = self.df.iloc[:self.n_days*24, out_features[0]].to_numpy()
        #print('gt.shape =', self.gt.shape)
        #print('n_days =', self.n_days)
        #print('n_days*24 =', self.n_days*24)
        
    def predict(self, model):
        preds = np.full((self.pred_days, self.n_days*24), np.nan)
        for i in tqdm(range(self.n_days-self.in_days)):
            x = self.df[i*24:(i*24+self.in_steps)].to_numpy()
            #x = val_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            y = model.predict(x)
            for j in range(self.pred_days):
                if i+self.in_days+j >= self.n_days:
                    break
                preds[j][(i+self.in_days+j)*24:(i+self.in_days+j+1)*24] = y[0, j*24:j*24+24, 0]
        return preds

In [37]:
def denormalize(data):
    return data*train_std[out_features[0]]+train_mean[out_features[0]]

In [38]:
preds_train = {}
preds_val = {}
preds_test = {}

**Daily avaraged prediction**

In [67]:
days_prediction_train = DaysPrediction(df=train_df)
days_prediction_val = DaysPrediction(df=val_df)
days_prediction_test = DaysPrediction(df=test_df)
daily = True

gt_train = denormalize(days_prediction_train.gt)
gt_val = denormalize(days_prediction_val.gt)
gt_test = denormalize(days_prediction_test.gt)

**Hourly prediction**

In [39]:
days_prediction_train = HoursPrediction(df=train_df)
days_prediction_val = HoursPrediction(df=val_df)
days_prediction_test = HoursPrediction(df=test_df)
daily = False

gt_train = denormalize(days_prediction_train.gt)
gt_val = denormalize(days_prediction_val.gt)
gt_test = denormalize(days_prediction_test.gt)

**Daily avaraged Model(warning)**

In [ ]:
days_prediction_train = Prediction(df=train_day_df)
days_prediction_val = Prediction(df=val_day_df)
days_prediction_test = Prediction(df=test_day_df)
daily = True

gt_train = denormalize(days_prediction_train.gt)
gt_val = denormalize(days_prediction_val.gt)
gt_test = denormalize(days_prediction_test.gt)

In [68]:
#preds_dict = {}
#days_prediction = DaysPrediction()
preds_train['GRU'] = denormalize(days_prediction_train.predict(multi_gru_model))
preds_val['GRU'] = denormalize(days_prediction_val.predict(multi_gru_model))
preds_test['GRU'] = denormalize(days_prediction_test.predict(multi_gru_model))

100%|██████████| 282/282 [00:38<00:00,  7.41it/s]


In [ ]:
preds_train['LSTM'] = denormalize(days_prediction_train.predict(multi_lstm_model))
preds_val['LSTM'] = denormalize(days_prediction_val.predict(multi_lstm_model))
preds_test['LSTM'] = denormalize(days_prediction_test.predict(multi_lstm_model))

In [ ]:
preds_train['Dense'] = denormalize(days_prediction_train.predict(multi_dense_model))
preds_val['Dense'] = denormalize(days_prediction_val.predict(multi_dense_model))
preds_test['Dense'] = denormalize(days_prediction_test.predict(multi_dense_model))

In [54]:
pred_day=4

In [69]:
plt.figure(figsize=(8, 3))
plt.plot(gt_train)
plt.plot(preds_train['GRU'][pred_day])
#plt.plot(preds_train['LSTM'][pred_day])
#plt.plot(preds_train['Dense'][pred_day])
plt.show()
plt.figure(figsize=(8, 3))
plt.plot(gt_val)
plt.plot(preds_val['GRU'][pred_day])
#plt.plot(preds_val['LSTM'][pred_day])
#plt.plot(preds_val['Dense'][pred_day])
plt.show()
plt.figure(figsize=(8, 3))
plt.plot(gt_test)
plt.plot(preds_test['GRU'][pred_day])
#plt.plot(preds_test['LSTM'][pred_day])
#plt.plot(preds_test['Dense'][pred_day])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
def daily_performance_metric(gt, pred):
    mae = np.mean(np.absolute(gt - pred))
    mape = np.mean(np.absolute(gt - pred)/gt)
    mean = np.mean(gt)
    nse = 1.-np.sum((gt - pred)**2) /  np.sum((gt - mean)**2)
    return { 'mae': mae, 'nse': nse}

In [57]:
daily_train_performance = {}
daily_val_performance = {}
daily_test_performance = {}

In [71]:
#model_names = ['GRU', 'LSTM', 'Dense']
model_names = ['GRU']
prefix = 'awsbo_'

if daily == True:
    pred_slice = slice(days_prediction_val.in_days+pred_day, None)
else:
    pred_slice = slice((days_prediction_val.in_days+pred_day)*24, None)

for model_name in model_names:
    daily_train_performance[prefix+model_name] = daily_performance_metric(gt_train[pred_slice], preds_train[model_name][pred_day][pred_slice])
    daily_val_performance[prefix+model_name] = daily_performance_metric(gt_val[pred_slice], preds_val[model_name][pred_day][pred_slice])
    daily_test_performance[prefix+model_name] = daily_performance_metric(gt_test[pred_slice], preds_test[model_name][pred_day][pred_slice])

In [72]:
daily_val_performance[prefix+'GRU']['nse']

0.5060862508056738

In [73]:
x = np.arange(len(daily_test_performance))
width = 0.3


#metric_name = 'mean_absolute_error'
#metric_index = multi_linear_model.metrics_names.index('mean_absolute_error')
metric_index = 'mae'
#metric_index = 'mae'
train_mae = [v[metric_index] for v in daily_train_performance.values()]
val_mae = [v[metric_index] for v in daily_val_performance.values()]
test_mae = [v[metric_index] for v in daily_test_performance.values()]
#test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.32, train_mae, width, label='Train')
plt.bar(x, val_mae, width, label='Validation')
plt.bar(x + 0.32, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=daily_test_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()


#metric_name = 'mean_absolute_error'
#metric_index = multi_linear_model.metrics_names.index('mean_absolute_error')
metric_index = 'nse'
#metric_index = 'mae'
train_nse = [v[metric_index] for v in daily_train_performance.values()]
val_nse = [v[metric_index] for v in daily_val_performance.values()]
test_nse = [v[metric_index] for v in daily_test_performance.values()]
#test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.32, train_nse, width, label='Train')
plt.bar(x, val_nse, width, label='Validation')
plt.bar(x + 0.32, test_nse, width, label='Test')
plt.xticks(ticks=x, labels=daily_test_performance.keys(),
           rotation=45)
plt.ylabel(f'NSE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
pred_day = 4
pred_slice = slice(days_prediction.in_days+pred_day, None)

In [ ]:
performance = {}

In [ ]:
pred_day = 4
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['GRU'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)
mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['LSTM'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['GRU'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)
mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['LSTM'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['GRU'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)
mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['LSTM'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae = np.mean(np.absolute(gt[pred_slice] - (denorm_preds['GRU'][pred_day])[pred_slice]))
print(mae)
mape = np.mean(np.absolute((gt[pred_slice] - denorm_preds['GRU'][pred_day][pred_slice])/gt[pred_slice]))
print(mape)
mean = np.mean(gt[pred_slice])
nse = 1.-np.sum((gt[pred_slice] - denorm_preds['GRU'][pred_day][pred_slice])**2) /  np.sum((gt[pred_slice] - mean)**2)
print('nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae = np.mean(np.absolute(gt[pred_slice] - (denorm_preds['LSTM'][pred_day])[pred_slice]))
print(mae)
mape = np.mean(np.absolute((gt[pred_slice] - denorm_preds['LSTM'][pred_day][pred_slice])/gt[pred_slice]))
print(mape)
mean = np.mean(gt[pred_slice])
nse = 1.-np.sum((gt[pred_slice] - denorm_preds['LSTM'][pred_day][pred_slice])**2) /  np.sum((gt[pred_slice] - mean)**2)
print('nse =', nse)

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
width=24*10
#width=3
channels=8
kernel_size=3
in_data = keras.layers.Input(shape=(width, channels))
x = in_data
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
#out_data = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
out_data = x

model = keras.models.Model(inputs=in_data, outputs=out_data)
model.summary()

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
8-7+1

In [ ]:
128*150

In [ ]:
train_df.shape